In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,Activation,LayerNormalization


%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import os,sys
import io


In [2]:
import sys 
sys.path.insert(0,"/content/drive/MyDrive/MLprojet/proj/projet/CNN")
%cd /content/drive/MyDrive/MLprojet/proj/projet/CNN
import function_training_CNN
from function_training_CNN import *
import function_submit_CNN
from function_submit_CNN import *


/content/drive/MyDrive/MLprojet/proj/projet/CNN


In [15]:
def load_image(infilename):
    data = mpimg.imread(infilename)
    return data 

def img_crop(im, patch_size, padding):
    """ From a test image, creates patches of height and width : window_size every patch_size pixels"""
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    #The test image needs to be padded to create windows with a center on the edge of the image
    im = np.lib.pad(im, ((padding, padding), (padding, padding), (0, 0)), 'reflect')
    for i in range(padding, imgheight + padding, patch_size):
        for j in range(padding, imgwidth + padding, patch_size):
            im_patch = im[j - padding:j + patch_size + padding, i - padding:i + patch_size + padding, :]
            list_patches.append(im_patch)
    return list_patches



def gen_patches(imgs, window_size, patch_size):
    """Generate patches from the set of test images"""
    padding_size = (window_size - patch_size) // 2
    patches = np.asarray(
        [img_crop(imgs[i], patch_size, padding_size) for i in  range(imgs.shape[0])])
    return patches.reshape(-1, patches.shape[2], patches.shape[3], patches.shape[4])




def mask_to_submission_strings_CNN(model,image_filename,window_size,patch_size,num_images=50):
    img_number = int(re.search(r"\d+", image_filename).group(0))
    image = load_image(image_filename)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    patches = gen_patches(image,window_size, patch_size)
    labels = model.predict(patches)
    labels = (labels[:,0]<labels[:,1])*1
    count = 0
    print("Processing image => " + image_filename)
    for j in range(0, image.shape[2], patch_size):
        for i in range(0, image.shape[1], patch_size):
            label = int(labels[count])
            count += 1
            yield ("{:03d}_{}_{},{}".format(img_number, j, i, label))

            
def generate_submission(model, submission_path, *image_filenames,window_size,patch_size):
    """ Generate a .csv containing the classification of the test set. """
    with open(submission_path, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames[0:]:
             f.writelines('{}\n'.format(s) for s in mask_to_submission_strings_CNN(model, fn,window_size,patch_size))

In [3]:
import os

from google.colab import drive
drive.mount('/content/drive')


root_dir = "/content/drive/MyDrive/MLprojet/proj/projet/training/"
#aug_5rot_Gblur5_Flips_Ligths/"
print(root_dir)


def load_image(infilename):
    data = mpimg.imread(infilename)
    return data 

#root_dir = "../training/"
image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n=100


print("Loading " + str(n) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
print(files[0])

gt_imgs = np.array(gt_imgs)
imgs = np.array(imgs)






Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/MLprojet/proj/projet/training/
Loading 100 images
Loading 100 images
satImage_001.png


In [7]:
class Cnn(keras.Model):

    def __init__(self):
        super(Cnn, self).__init__()
        
        keras.backend.set_image_data_format('channels_last')

        self.patch_size = 16
        self.window_size = 80 #chosen such that enough context surround a patch of size 16
        self.nb_channels = 3 
        self.nb_classes = 2  
        self.batch_size = 50
        self.model()
        

    def layer(self,depth,filter):
        self.model.add(layers.Conv2D(depth, filter, padding='same'))
        self.model.add(layers.LeakyReLU(alpha=0.1))
        self.model.add(layers.MaxPool2D(padding='same'))
        self.model.add(layers.Dropout(0.25))

    def get_f1( y_true, y_pred): #taken from old keras source code
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        recall = true_positives / (possible_positives + K.epsilon())
        f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
        return f1_val
  
    def model(self):
       
        self.model = keras.Sequential()

        # Input layer
        input_shape = (self.window_size, self.window_size, self.nb_channels)
        self.model.add(layers.InputLayer(input_shape))

        #First layer 
        self.layer(64,5)
        

        #Second layer 
        self.layer(128,3)

        # Third layer 
        self.layer(256,3)
        self.model.add(layers.Flatten())

        # Fourth fully connected layer 
        self.model.add(layers.Dense(128, kernel_regularizer=keras.regularizers.l2(1e-6)))
        self.model.add(layers.LeakyReLU(alpha=0.1))
        self.model.add(layers.Dropout(0.5))

        # Softmax activation function
        self.model.add(
            layers.Dense(self.nb_classes, kernel_regularizer=keras.regularizers.l2(1e-6),
                         activation='softmax'))

        # Adam optimizer
        optimizer = keras.optimizers.Adam()

        # Binary cross_entropy loss
        self.model.compile(optimizer=optimizer, loss=keras.losses.binary_crossentropy,
                           metrics=[get_f1])
        
        self.model.summary()

    def train_model(self, gt_imgs, imgs, nb_epochs):
        """Trains the CNN model """

        padding_size = (self.window_size - self.patch_size) // 2 

        #Padding of the image
        imgs = np.asarray([pad(imgs[i], padding_size) for i in range(imgs.shape[0])])
        gt_imgs = np.asarray([pad(gt_imgs[i], padding_size) for i in range(imgs.shape[0])])
        
        
        history = self.model.fit_generator(batches_generator(imgs, gt_imgs, self.batch_size, self.window_size,self.patch_size,self.nb_classes),
                                               steps_per_epoch=30,
                                               epochs=nb_epochs,
                                               verbose=1)
        
        
        return history
    
    def get_f1(self,  y_true, y_pred): #taken from old keras source code
       true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
       possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
       predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
       precision = true_positives / (predicted_positives + K.epsilon())
       recall = true_positives / (possible_positives + K.epsilon())
       f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
       return f1_val

    def predict(self,
                x,
                batch_size=None,
                verbose=0,
                steps=None,
                callbacks=None,
                max_queue_size=10,
                workers=1,
                use_multiprocessing=False):
          labels = self.model.predict(x)
          return labels
          

 

    
    

In [8]:
cnn = Cnn()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 80, 80, 64)        4864      
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 80, 80, 64)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 40, 40, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 40, 40, 64)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 40, 40, 128)       73856     
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 40, 40, 128)       0         
                                                      

In [9]:
 history = cnn.train_model(gt_imgs, imgs,80)
 filepath = "/content/drive/MyDrive/MLprojet/proj/projet/Saved_models/final_notaug.h5"

cnn.save_weights(filepath)
print("model saved !")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/80
 2/30 [=>............................] - ETA: 58s - loss: 1.2802 - get_f1: 0.6400 

KeyboardInterrupt: ignored

In [12]:

#Load test images
root_dir = "/content/drive/MyDrive/MLprojet/proj/projet/test_set/"

image_dir = root_dir + "images/"
image_filenames_test = [image_dir + 'test_' + str(i + 1)+'.png' for i in range(50)]

print(image_filenames_test[0])



/content/drive/MyDrive/MLprojet/proj/projet/test_set/images/test_1.png


In [13]:
filepath = "/content/drive/MyDrive/MLprojet/proj/projet/Saved_models/final_notaug.h5"
cnn =Cnn()
cnn.built = True
cnn.load_weights(filepath)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 80, 80, 64)        4864      
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 80, 80, 64)        0         
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 40, 40, 64)       0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 40, 40, 64)        0         
                                                                 
 conv2d_10 (Conv2D)          (None, 40, 40, 128)       73856     
                                                                 
 leaky_re_lu_13 (LeakyReLU)  (None, 40, 40, 128)       0         
                                                      

In [14]:
import re
submission_path = '/content/drive/MyDrive/MLprojet/proj/projet/\submission_unet.csv'
generate_submission(cnn, submission_path, *image_filenames_test,window_size=80,patch_size=16)

Processing image => /content/drive/MyDrive/MLprojet/proj/projet/test_set/images/test_1.png
Processing image => /content/drive/MyDrive/MLprojet/proj/projet/test_set/images/test_2.png


KeyboardInterrupt: ignored